In [1]:
import numpy as np
import pygame, sys
from pygame.locals import *

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
WIDTH = HEIGHT = 576
    
DIMENSION = 9
    
SQ_SIZE = HEIGHT // DIMENSION
    
MAX_FPS = 15
WHITE=(255,255,255)
BLUE=(0,0,255)
RED = (255, 0, 0)
BLACK = (0,0,0)

# can rotate all pieces with np.rot90(m, k=1)

# m 	Array of two or more dimensions. 
# k 	Number of times the array is rotated by 90 degrees. 


pieces = [
        np.array([[1, 1],
                  [1, 1]]), # 0

        np.array([[0, 1, 0],
                  [1, 1, 1]]), # 1

        np.array([[0, 1, 1],
                  [1, 1, 0]]), # 2

        np.array([[0, 1, 0],
                  [0, 1, 0],
                  [1, 1, 1]]), # 3

        np.array([[1, 1, 1, 1]]), # 4

        np.array([[1, 1, 1]]), # 5

        np.array([[0, 1],
                  [1, 1]]), # 6

        np.array([[1, 0, 0],
                  [1, 1, 1]]), # 7

        np.array([[1, 0, 0],
                  [1, 0, 0],
                  [1, 1, 1]]), # 8
        
        np.array([[1, 1]]), # 9
    
        np.array([[1, 0, 0],
                  [0, 1, 0],
                  [0, 0, 1]]), # 10
    
        np.array([[1, 0],
                  [0, 1]]) # 11
    
    ]

In [3]:
class GameState():
    def __init__(self, screen, all_sprites):
        self.board = np.zeros((9,9), dtype = int)
        
        # assign random piece indices
        p = np.random.rand(3)
        
        pr = []
        y_off = 0
        # make piece orientation random
        for i in p:
            rotated = np.rot90(pieces[int(round(i*9))], k=int(round(np.random.rand()*9)))
            piece_object = Piece(piece=rotated,clicked=0,placed = 0, screen=screen, y_off = y_off)
            all_sprites.add(piece_object)
            pr.append(piece_object)
            y_off += (1+rotated.shape[0])*SQ_SIZE
         
        # piece is object with a piece and a flag if placed
        self.pcs = pr
        self.lose = False

In [4]:
def drawUnplaced(screen,gs,square):
    y_offset = 0
    for piece,flag in gs.pcs:
        x,y = piece.shape
        if flag == 0:
            for r in range(x):
                for c in range(y):
                    if piece[r][c] == 1:
                        screen.blit(square,((DIMENSION+1)*SQ_SIZE+c*SQ_SIZE,r*SQ_SIZE + y_offset, SQ_SIZE-2, SQ_SIZE-2))      
        y_offset += (1+x)*SQ_SIZE

In [5]:
class Piece(pygame.sprite.Sprite): 
    def __init__(self, piece, clicked, placed, screen, y_off):
        pygame.sprite.Sprite.__init__(self)
        self.piece = piece
        self.clicked = clicked
        self.placed = placed
        self.y_off = y_off
        self.screen = screen
        r,c = self.piece.shape
        
        self.image = pygame.Surface((c*(SQ_SIZE-2),r*(SQ_SIZE-2)),pygame.SRCALPHA, 32)
        self.image.convert_alpha()

        for y in range(r):
            for x in range(c):
                if piece[y][x] == 1:
                    self.image.fill(BLUE,(x*SQ_SIZE,y*SQ_SIZE,SQ_SIZE-2, SQ_SIZE-2))
                    
        self.rect = self.image.get_rect()       
        self.rect.x = (DIMENSION+1)*SQ_SIZE
        self.rect.y = self.y_off
        
        

        
    def move(self, rel_x, rel_y):
        self.rect.x += rel_x
        self.rect.y += rel_y
                    
        
    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            location = pygame.mouse.get_pos() # returns x,y location of the mouse
            # if mouse press is on board disregard
            if location[0]< SQ_SIZE*(DIMENSION+1):
                return
            elif location[1] < self.y_off:
                return
            elif location[1] > (self.y_off + (1+self.piece.shape[0])*SQ_SIZE):
                return
                
            mx, my = location[0]//SQ_SIZE, location[1]//SQ_SIZE
            y_offset = self.y_off//SQ_SIZE
            r,c = self.piece.shape
            for i in range(r):
                for j in range(c):
                    if self.piece[i][j] == 1:
                        if mx == ((DIMENSION+1) + j):
                            if my == y_offset + i:
                                #drag from piece at ij as center
                                #piece_draging = True
                                #mouse_x, mouse_y = event.pos
                                # get piece object function
                                #offset_x = rectangle.x - mouse_x
                                #offset_y = rectangle.y - mouse_y
                                self.clicked = 1
                        # if this ends then not the first piece so update to next y piece
        
        elif event.type == pygame.MOUSEBUTTONUP:
            if self.clicked:
                #place piece
                if not self.placed:
                    self.rect.x = (DIMENSION+1)*SQ_SIZE
                    self.rect.y = self.y_off
                    
                self.clicked = 0
        elif event.type == pygame.MOUSEMOTION:
            if self.clicked:
                self.move(event.rel[0], event.rel[1])
                
                

In [6]:
hello = 3


In [7]:
def drawGameState(screen,gs, square):
    drawBoard(screen)
    drawPlaced(screen,gs.board, square)
    #drawUnplaced(screen,gs,square)

In [8]:
def drawBoard(screen):
    sudoku_x_offset = 0
    sudoku_y_offset = 0
    for r in range(DIMENSION):
        if (r%3) == 0:
                sudoku_y_offset += 3
                
        for c in range(DIMENSION):
            if (c%3) == 0:
                sudoku_x_offset += 3
            pygame.draw.rect(screen, BLACK, (c*SQ_SIZE+sudoku_x_offset,r*SQ_SIZE+sudoku_y_offset, SQ_SIZE+6, SQ_SIZE+6))
            pygame.draw.rect(screen, WHITE, (c*SQ_SIZE+1+sudoku_x_offset,r*SQ_SIZE+1+sudoku_y_offset, SQ_SIZE-2, SQ_SIZE-2))
            
        sudoku_x_offset = 0

In [9]:
def drawPlaced(screen,board,square):
    for r in range(DIMENSION):
        for c in range(DIMENSION):
            if board[r][c] == 1:
                screen.blit(square,(c*SQ_SIZE,r*SQ_SIZE, SQ_SIZE, SQ_SIZE))

In [10]:
def loadSquare():
        square = pygame.transform.scale(pygame.image.load("images\\blue.jpg"),(SQ_SIZE,SQ_SIZE))
        return square

In [11]:
def calculatePoints(gs):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-11-d3e59ab3e612>, line 2)

In [12]:
def placePiece(index, board, location, gs):
    temp = board.copy()
    #print(reset)
    piece = gs.pcs[index][0]
    
    r,c = piece.shape
    
    pr,pc = location
    
    for i in range(r):
        for j in range(c):
            if (piece[i][j] == 1):
                if (pr + i > DIMENSION - 1 or pc + j > DIMENSION - 1 or temp[pr+i][pc+j] == 1):
                    print("invalid spot")
                    return              
                if (temp[pr+i][pc+j] == 0):
                    temp[pr+i][pc+j] = 1
    board = temp
    
    # set as placed
    gs.pcs[index][1] = 1
    print('success')
    return
                

In [13]:
def main():
    pygame.init()
    
    DISPLAY=pygame.display.set_mode((896,800))
    DISPLAY.fill(WHITE)
    
    # set the pygame window name 
    pygame.display.set_caption('Game Window') 
      
    # create a font object. 
    # 1st parameter is the font file 
    # which is present in pygame. 
    # 2nd parameter is size of the font 
    font = pygame.font.Font('freesansbold.ttf', 20) 
    all_sprites = pygame.sprite.Group()

    gs = GameState(DISPLAY, all_sprites)
    square = loadSquare()
    running = True
    print(gs.pcs)
    

    while running:
        for event in pygame.event.get():
            if event.type==QUIT:
                pygame.quit()
                sys.exit()
                
            for o in gs.pcs:
                o.handle_event(event)
        
        DISPLAY.fill(WHITE)
        drawGameState(DISPLAY,gs, square)
        all_sprites.update()
        all_sprites.draw(DISPLAY)
        
        #points = calculatePoints(gs)
        score = "Score: " + str(300)
        
        # create a text suface object, 
        # on which text is drawn on it. 
        text = font.render(score, True, BLACK) 
      
        # create a rectangular object for the 
        # text surface object 
        textRect = text.get_rect()  
      
        # set the center of the rectangular object. 
        textRect.center = (288, 676)
        DISPLAY.blit(text, textRect)
        pygame.display.update()

main()

[<Piece sprite(in 1 groups)>, <Piece sprite(in 1 groups)>, <Piece sprite(in 1 groups)>]


SystemExit: 

C:\Users\Manuel Lopez\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [34]:
# can rotate all pieces with np.rot90(m, k=1)

# m 	Array of two or more dimensions. 
# k 	Number of times the array is rotated by 90 degrees. 


pieces = [
        np.array([[1, 1],
                  [1, 1]]),

        np.array([[0, 1, 0],
                  [1, 1, 1]]),

        np.array([[0, 1, 1],
                  [1, 1, 0]]),

        np.array([[0, 1, 0],
                  [0, 1, 0],
                  [1, 1, 1]]),

        np.array([[1, 1, 1, 1]]),

        np.array([[1, 1, 1]]),

        np.array([[0, 1],
                  [1, 1]]),

        np.array([[1, 0, 0],
                  [1, 1, 1]]),

        np.array([[1, 0, 0],
                  [1, 0, 0],
                  [1, 1, 1]]),
        
        np.array([[1, 1]])
    ]

In [78]:
gs = GameState()

In [48]:
gs.board

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [81]:
def placePiece(index, board, location, gs):
    temp = board.copy()
    #print(reset)
    piece = gs.pcs[index][0]
    
    r,c = piece.shape
    
    pr,pc = location
    
    for i in range(r):
        for j in range(c):
            if (piece[i][j] == 1):
                if (pr + i > DIMENSION - 1 or pc + j > DIMENSION - 1 or temp[pr+i][pc+j] == 1):
                    print("invalid spot")
                    return              
                if (temp[pr+i][pc+j] == 0):
                    temp[pr+i][pc+j] = 1
    board = temp
    
    # set as placed
    gs.pcs[index][1] = 1
    return
                

In [4]:
testboard = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 1, 0, 0, 0, 0, 1, 0, 0],
                      [0, 0, 1, 1, 0, 0, 1, 0, 1],
                      [0, 1, 1, 0, 1, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0, 1, 1, 1],
                      [0, 0, 0, 1, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [5]:
p = pieces[7]

p = np.rot90(p, k=2)

In [6]:
loc = 3,0

In [217]:
placePiece(p, testboard, loc)

invalid spot


In [218]:
testboard

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 1, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [210]:
p

array([[1, 1, 1],
       [0, 0, 1]])

In [192]:
loc

(3, 0)

In [7]:
piece1 = p, 0

In [10]:
ps = [piece1]


In [11]:
ps.append(piece1)

In [12]:
ps

[(array([[1, 1, 1],
         [0, 0, 1]]), 0), (array([[1, 1, 1],
         [0, 0, 1]]), 0)]

In [80]:
gs.pcs[2][0]

array([[1, 0],
       [1, 1],
       [1, 0]])

In [5]:
! git add .

The file will have its original line endings in your working directory
The file will have its original line endings in your working directory


In [6]:
! git commit -m "test"

[master cbca327] test
 2 files changed, 819 insertions(+)
 create mode 100644 src/.ipynb_checkpoints/Blockudoku-checkpoint.ipynb


In [7]:
! git push

To https://github.com/mslopezg/blockudokuAI.git
   ec09b99..cbca327  master -> master
